Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/reinforcement-learning/cartpole-on-compute-instance/cartpole_ci.png)

# Reinforcement Learning in Azure Machine Learning - Cartpole Problem on Compute Instance

Reinforcement Learning in Azure Machine Learning is a managed service for running reinforcement learning training and simulation. With Reinforcement Learning in Azure Machine Learning, data scientists can start developing reinforcement learning systems on one machine, and scale to compute targets with 100s of nodes if needed.

This example shows how to use Reinforcement Learning in Azure Machine Learning to train a Cartpole playing agent on a compute instance.

### Cartpole problem

Cartpole, also known as [Inverted Pendulum](https://en.wikipedia.org/wiki/Inverted_pendulum), is a pendulum with a center of mass above its pivot point. This formation is essentially unstable and will easily fall over but can be kept balanced by applying appropriate horizontal forces to the pivot point.

<table style="width:50%">
  <tr>
    <th>
      <img src="./images/cartpole.png" alt="Cartpole image" /> 
    </th>
  </tr>
  <tr>
      <th><p>Fig 1. Cartpole problem schematic description (from <a href="https://towardsdatascience.com/cartpole-introduction-to-reinforcement-learning-ed0eb5b58288">towardsdatascience.com</a>).</p></th>
  </tr>
</table>

The goal here is to train an agent to keep the cartpole balanced by applying appropriate forces to the pivot point.

See [this video](https://www.youtube.com/watch?v=XiigTGKZfks) for a real-world demonstration of cartpole problem.

### Prerequisite
The user should have completed the Azure Machine Learning Tutorial: [Get started creating your first ML experiment with the Python SDK](https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-1st-experiment-sdk-setup). You will need to make sure that you have a valid subscription ID, a resource group, and an Azure Machine Learning workspace. All datastores and datasets you use should be associated with your workspace.

## Set up Development Environment
The following subsections show typical steps to setup your development environment. Setup includes:

* Connecting to a workspace to enable communication between your local machine and remote resources
* Creating an experiment to track all your runs
* Using a Compute Instance as compute target

### Azure Machine Learning SDK 
Display the Azure Machine Learning SDK version.

In [ ]:
import azureml.core
print("Azure Machine Learning SDK Version:", azureml.core.VERSION)

### Get Azure Machine Learning workspace
Get a reference to an existing Azure Machine Learning workspace.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep = ' | ')

### Use  Compute Instance as compute target

A compute target is a designated compute resource where you run your training and simulation scripts. This location may be your local machine or a cloud-based compute resource. For more information see [What are compute targets in Azure Machine Learning?](https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target)

The code below shows how to use current compute instance as a compute target. First some helper functions:

In [ ]:
import os.path

# Get information about the currently running compute instance (notebook VM), like its name and prefix.
def load_nbvm():
    if not os.path.isfile("/mnt/azmnt/.nbvm"):
        return None
    with open("/mnt/azmnt/.nbvm", 'r') as nbvm_file:
        return { key:value for (key, value) in [ line.strip().split('=') for line in nbvm_file if '=' in line ] }


Then we use these helper functions to get a handle to current compute instance.

In [ ]:
from azureml.core.compute import ComputeInstance
from azureml.core.compute_target import ComputeTargetException

import random
import string

# Load current compute instance info
current_compute_instance = load_nbvm()

# For this demo, let's use the current compute instance as the compute target, if available
if current_compute_instance:
    print("Current compute instance:", current_compute_instance)
    instance_name = current_compute_instance['instance']
else:
    # Compute instance name needs to be unique across all existing compute instances within an Azure region
    instance_name = "cartpole-ci-" + "".join(random.choice(string.ascii_lowercase) for _ in range(5))
    try:
        instance = ComputeInstance(workspace=ws, name=instance_name)
        print('Found existing instance, use it.')
    except ComputeTargetException:
        print("Creating new compute instance...")
        compute_config = ComputeInstance.provisioning_configuration(
            vm_size='STANDARD_D2_V2'
        )
        instance = ComputeInstance.create(ws, instance_name, compute_config)
        instance.wait_for_completion(show_output=True)
    print("Instance name:", instance_name)

compute_target = ws.compute_targets[instance_name]

print("Compute target status:")
print(compute_target.get_status().serialize())

### Create Azure Machine Learning experiment
Create an experiment to track the runs in your workspace. 

In [ ]:
from azureml.core.experiment import Experiment

experiment_name = 'CartPole-v0-CI'
exp = Experiment(workspace=ws, name=experiment_name)

## Train Cartpole Agent
To facilitate reinforcement learning, Azure Machine Learning Python SDK provides a high level abstraction, the _ReinforcementLearningEstimator_ class, which allows users to easily construct reinforcement learning run configurations for the underlying reinforcement learning framework. Reinforcement Learning in Azure Machine Learning supports the open source [Ray framework](https://ray.io/) and its highly customizable [RLlib](https://ray.readthedocs.io/en/latest/rllib.html#rllib-scalable-reinforcement-learning). In this section we show how to use _ReinforcementLearningEstimator_ and Ray/RLlib framework to train a cartpole playing agent. 

### Create reinforcement learning estimator

The code below creates an instance of *ReinforcementLearningEstimator*, `training_estimator`, which then will be used to submit a job to Azure Machine Learning to start the Ray experiment run.

Note that this example is purposely simplified to the minimum. Here is a short description of the parameters we are passing into the constructor:

- `source_directory`, local directory containing your training script(s) and helper modules,
- `entry_script`, path to your entry script relative to the source directory,
- `script_params`, constant parameters to be passed to each run of training script,
- `compute_target`, reference to the compute target in which the trainer and worker(s) jobs will be executed,
- `rl_framework`, the reinforcement learning framework to be used (currently must be Ray).

We use the `script_params` parameter to pass in general and algorithm-specific parameters to the training script.


In [ ]:
from azureml.contrib.train.rl import ReinforcementLearningEstimator, Ray

training_algorithm = "PPO"
rl_environment = "CartPole-v0"

script_params = {

    # Training algorithm
    "--run": training_algorithm,
    
    # Training environment
    "--env": rl_environment,
    
    # Algorithm-specific parameters
    "--config": '\'{"num_gpus": 0, "num_workers": 1}\'',
    
    # Stop conditions
    "--stop": '\'{"episode_reward_mean": 200, "time_total_s": 300}\'',
    
    # Frequency of taking checkpoints
    "--checkpoint-freq": 2,
    
    # If a checkpoint should be taken at the end - optional argument with no value
    "--checkpoint-at-end": "",
    
    # Log directory
    "--local-dir": './logs'
}

training_estimator = ReinforcementLearningEstimator(

    # Location of source files
    source_directory='files',
    
    # Python script file
    entry_script='cartpole_training.py',
    
    # A dictionary of arguments to pass to the training script specified in ``entry_script``
    script_params=script_params,
    
    # The Azure Machine Learning compute target set up for Ray head nodes
    compute_target=compute_target,
    
    # Reinforcement learning framework. Currently must be Ray.
    rl_framework=Ray()
)

### Training script

As recommended in RLlib documentations, we use Ray Tune API to run the training algorithm. All the RLlib built-in trainers are compatible with the Tune API. Here we use `tune.run()` to execute a built-in training algorithm. For convenience, down below you can see part of the entry script where we make this call.

This is the list of parameters we are passing into `tune.run()` via the `script_params` parameter:

- `run_or_experiment`: name of the [built-in algorithm](https://ray.readthedocs.io/en/latest/rllib-algorithms.html#rllib-algorithms), 'PPO' in our example,
- `config`: Algorithm-specific configuration. This includes specifying the environment, `env`, which in our example is the gym **[CartPole-v0](https://gym.openai.com/envs/CartPole-v0/)** environment,
- `stop`: stopping conditions, which could be any of the metrics returned by the trainer. Here we use "mean of episode reward", and "total training time in seconds" as stop conditions, and
- `checkpoint_freq` and `checkpoint_at_end`: Frequency of taking checkpoints (number of training iterations between checkpoints), and if a checkpoint should be taken at the end.

We also specify the `local_dir`, the directory in which the training logs, checkpoints and other training artificats will be recorded. 

See [RLlib Training APIs](https://ray.readthedocs.io/en/latest/rllib-training.html#rllib-training-apis) for more details, and also [Training (tune.run, tune.Experiment)](https://ray.readthedocs.io/en/latest/tune/api_docs/execution.html#training-tune-run-tune-experiment) for the complete list of parameters.

```python
import ray
import ray.tune as tune

if __name__ == "__main__":

    # parse arguments ...
    
    # Intitialize ray
    ay.init(address=args.ray_address)

    # Run training task using tune.run
    tune.run(
        run_or_experiment=args.run,
        config=dict(args.config, env=args.env),
        stop=args.stop,
        checkpoint_freq=args.checkpoint_freq,
        checkpoint_at_end=args.checkpoint_at_end,
        local_dir=args.local_dir
    )
```

### Submit the estimator to start experiment
Now we use the *training_estimator* to submit a run.

In [ ]:
training_run = exp.submit(training_estimator)

### Monitor experiment
Azure Machine Learning provides a Jupyter widget to show the status of an experiment run. You could use this widget to monitor the status of the runs.

Note that _ReinforcementLearningEstimator_ creates at least two runs: (a) A parent run, i.e. the run returned above, and (b) a collection of child runs. The number of the child runs depends on the configuration of the reinforcement learning estimator. In our simple scenario, configured above, only one child run will be created.

The widget will show a list of the child runs as well. You can click on the link under **Status** to see the details of a child run. It will also show the metrics being logged.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(training_run).show()

### Stop the run

To stop the run, call `training_run.cancel()`.

In [ ]:
# Uncomment line below to cancel the run
# training_run.cancel()

### Wait for completion
Wait for the run to complete before proceeding.

**Note: The run may take a few minutes to complete.**

In [ ]:
training_run.wait_for_completion()

### Get a handle to the child run
You can obtain a handle to the child run as follows. In our scenario, there is only one child run, we have it called `child_run_0`.

In [ ]:
import time

child_run_0 = None
timeout = 30
while timeout > 0 and not child_run_0:
    child_runs = list(training_run.get_children())
    print('Number of child runs:', len(child_runs))
    if len(child_runs) > 0:
        child_run_0 = child_runs[0]
        break
    time.sleep(2) # Wait for 2 seconds
    timeout -= 2

print('Child run info:')
print(child_run_0)

## Evaluate Trained Agent and See Results

We can evaluate a previously trained policy using the `rollout.py` helper script provided by RLlib (see [Evaluating Trained Policies](https://ray.readthedocs.io/en/latest/rllib-training.html#evaluating-trained-policies) for more details). Here we use an adaptation of this script to reconstruct a policy from a checkpoint taken and saved during training. We took these checkpoints by setting `checkpoint-freq` and `checkpoint-at-end` parameters above.

In this section we show how to get access to these checkpoints data, and then how to use them to evaluate the trained policy.

### Create a dataset of training artifacts
To evaluate a trained policy (a checkpoint) we need to make the checkpoint accessible to the rollout script.
We can use the Run API to download policy training artifacts (saved model and checkpoints) to local compute.

In [ ]:
from os import path
from distutils import dir_util

training_artifacts_path = path.join("logs", training_algorithm)
print("Training artifacts path:", training_artifacts_path)

if path.exists(training_artifacts_path):
    dir_util.remove_tree(training_artifacts_path)

# Download run artifacts to local compute
child_run_0.download_files(training_artifacts_path)

Now let's find the checkpoints and the last checkpoint number.

In [ ]:
# A helper function to find checkpoint files in a directory
def find_checkpoints(file_path):
    print("Looking in path:", file_path)
    checkpoints = []
    for root, _, files in os.walk(file_path):
        for name in files:
            if os.path.basename(root).startswith('checkpoint_'):
                checkpoints.append(path.join(root, name))
    return checkpoints

In [ ]:
# Find checkpoints and last checkpoint number
checkpoint_files = find_checkpoints(training_artifacts_path)

checkpoint_numbers = []
for file in checkpoint_files:
    file = os.path.basename(file)
    if file.startswith('checkpoint-') and not file.endswith('.tune_metadata'):
        checkpoint_numbers.append(int(file.split('-')[1]))

print("Checkpoints:", checkpoint_numbers)

last_checkpoint_number = max(checkpoint_numbers)
print("Last checkpoint number:", last_checkpoint_number)

Now we upload checkpoints to default datastore and create a file dataset. This dataset will be used to pass in the checkpoints to the rollout script.

In [ ]:
# Upload the checkpoint files and create a DataSet
from azureml.core import Dataset

datastore = ws.get_default_datastore()
checkpoint_dataref = datastore.upload_files(checkpoint_files, target_path='cartpole_checkpoints_' + run_id, overwrite=True)
checkpoint_ds = Dataset.File.from_files(checkpoint_dataref)

To verify, we can print out the number (and paths) of all the files in the dataset.

In [ ]:
artifacts_paths = checkpoint_ds.to_path()
print("Number of files in dataset:", len(artifacts_paths))

# Uncomment line below to print all file paths
#print("Artifacts dataset file paths: ", artifacts_paths)

### Evaluate a trained policy
We need to configure another reinforcement learning estimator, `rollout_estimator`, and then use it to submit another run. Note that the entry script for this estimator now points to `cartpole-rollout.py` script.
Also note how we pass the checkpoints dataset to this script using `inputs` parameter of the _ReinforcementLearningEstimator_.

We are using script parameters to pass in the same algorithm and the same environment used during training. We also specify the checkpoint number of the checkpoint we wish to evaluate, `checkpoint-number`, and number of the steps we shall run the rollout, `steps`.

The checkpoints dataset will be accessible to the rollout script as a mounted folder. The mounted folder and the checkpoint number, passed in via `checkpoint-number`, will be used to create a path to the checkpoint we are going to evaluate. The created checkpoint path then will be passed into RLlib rollout script for evaluation.

Now let's configure rollout estimator. Note that we use the last checkpoint for evaluation. The assumption is that the last checkpoint points to our best trained agent. You may change this to any of the checkpoint numbers printed above and observe the effect.

In [ ]:
script_params = {    
    # Checkpoint number of the checkpoint from which to roll out
    "--checkpoint-number": last_checkpoint_number,

    # Training algorithm
    "--run": training_algorithm,
    
    # Training environment
    "--env": rl_environment,
    
    # Algorithm-specific parameters
    "--config": '{}',
    
    # Number of rollout steps 
    "--steps": 2000,
    
    # If should repress rendering of the environment
    "--no-render": ""
}

rollout_estimator = ReinforcementLearningEstimator(
    # Location of source files
    source_directory='files',
    
    # Python script file
    entry_script='cartpole_rollout.py',
    
    # A dictionary of arguments to pass to the rollout script specified in ``entry_script``
    script_params = script_params,
    
    # Data inputs
    inputs=[
        checkpoint_ds.as_named_input('artifacts_dataset'),
        checkpoint_ds.as_named_input('artifacts_path').as_mount()],
    
    # The Azure Machine Learning compute target
    compute_target=compute_target,
    
    # Reinforcement learning framework. Currently must be Ray.
    rl_framework=Ray(),
    
    # Additional pip packages to install
    pip_packages = ['azureml-dataset-runtime[fuse,pandas]'])

Same as before, we use the *rollout_estimator* to submit a run.

In [ ]:
rollout_run = exp.submit(rollout_estimator)

And then, similar to the training section, we can monitor the real-time progress of the rollout run and its chid as follows. If you browse logs of the child run you can see the evaluation results recorded in driver_log.txt file. Note that you may need to wait several minutes before these results become available.

In [ ]:
RunDetails(rollout_run).show()

Wait for completion of the rollout run, or you may cancel the run.

In [ ]:
# Uncomment line below to cancel the run
#rollout_run.cancel()
rollout_run.wait_for_completion()

## Cleaning up
For your convenience, below you can find code snippets to clean up any resources created as part of this tutorial that you don't wish to retain.

In [ ]:
# To archive the created experiment:
#exp.archive()

# To delete created compute instance
if not current_compute_instance:
    compute_target.delete()

## Next
This example was about running Reinforcement Learning in Azure Machine Learning (Ray/RLlib Framework) on a compute instance. Please see [Cartpole Problem on Single Compute](../cartpole-on-single-compute/cartpole_sc.ipynb)
example which uses Ray RLlib to train a Cartpole playing agent on a single node remote compute.
